In [4]:
import pandas as pd
import math
import numpy as np

In [5]:
# Definiujemy funkcje aktywacji fi (sigmoida)
# fi' = fi*(1-fi) 

def sigmoid(x): return (1+np.exp(-x))**(-1) 

def deriv_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))

# Odwrotna funkcja sigmoidalna - Czy można jej tutaj użyć?
def inv_sigmoid(x): return np.log(x/(1-x))

In [6]:
# Generujemy wagi
def wygeneruj_wagi(wejscie, wyjscie):
    # wyjściem jest liczba neuronów w następnej warstwie
    wektor_wag = np.random.normal(0,1/math.sqrt(len(wejscie)),(1+len(wejscie))*wyjscie)
    return np.reshape(wektor_wag, (wyjscie, 1+len(wejscie) )  )

In [3]:
wygeneruj_wagi((1,2), 2)

NameError: name 'math' is not defined

In [ ]:
# Wejscie = wektor 784
# Wyjscie = wektor 10

# nerunony_na_warstwe: podajemy liczbę na warstwy głębokie
def forward_prop(wejscie_dane, liczb_neuronow_wyjscie ):
    """Podajemy dane na wejście, dodajemy na bias = 1 na początek wektora (otrzymujemy w ten sposób X).
    Zadajemy najpierw ile chcemy mieć neuronów na wyjściu
    Nastepnie liczymy net = W*X <- wyjście netto.
    Póżniej nakładamy funkcję aktywacji fi na net, otrzymując fi(net) = a <- wyjście z warstwy.
    Musimy też zapisywać stany x, net, a dla każdej warstwy (np. lista krotek)"""

    stany_x_net_a = []
    # Musimy przerobić X na wektor kolumnowy
    X = np.vstack([1,wejscie_dane.reshape(-1,1)])
    W = wygeneruj_wagi(wejscie_dane, liczb_neuronow_wyjscie)
    net = W @ X
    a = sigmoid(net)
    stany_x_net_a.append((X,net,a))
    return stany_x_net_a


In [ ]:
A1 = forward_prop(np.array([1,1,0]), liczb_neuronow_wyjscie=2)[0][2]
print(A1)

A2 = forward_prop(A1, liczb_neuronow_wyjscie=2)[0][2]
print(A2)

In [ ]:
# Definiujemy propagację wstecz

# wyjscie oczekiwane - to co chcielibyśmy uzyskać
# wyjscie dane - nasz wynik z forward_prop

# Funkcja straty to L = 1/2(a[L]-y)^2, czyli pochodna z L to a[L]-y
def back_prop(wyjscie_oczekiwane, wyjscie_dane, wagi_warstwy, stala_uczenia = 0.01 ):

    # Ostatnia warstwa
    a_ost = 
    dL_a_wyjscia = wyjscie_dane - wyjscie_oczekiwane

    # mamy pochodną dL/da * fi(net)
    delta = dL_a_wyjscia * deriv_sigmoid( inv_sigmoid(wyjscie_dane) )
    dL_dW = delta * wyjscie_dane.T

    # Liczymy dL/dX i usuwamy pierwszy element, otrzymując dL/da niższej warstwy
    dL_dX = wagi_warstwy.T @ delta

    dL_da_nizsze = dL_dX[1:]

    # Aktualizujemy wagi
    W_nowe = wagi_warstwy - stala_uczenia*dL_dW


In [ ]:
A = np.array([[0.1,-0.2,0.3], [-0.4,0.5,-0.6]])
B = np.array([1,1,0])
C= B.reshape(-1,1)
C

In [ ]:
A@B

In [ ]:
np.hstack([1,A])

In [ ]:
sigmoid(C)

In [ ]:
import numpy as np
import pickle
import gzip

class Warstwa:
    def __init__(self, liczba_wejsc, liczba_neuronow, funkcja_aktywacji='sigmoid'):
        self.liczba_wejsc = liczba_wejsc
        self.liczba_neuronow = liczba_neuronow
        
        # Inicjalizacja wag - Xavier/Glorot initialization
        self.wagi = np.random.randn(liczba_neuronow, liczba_wejsc + 1) * np.sqrt(2.0 / (liczba_wejsc + liczba_neuronow))
        
        self.funkcja_aktywacji = funkcja_aktywacji
        self.wejscie = None
        self.net = None
        self.wyjscie = None
        
    def aktywacja(self, x):
        if self.funkcja_aktywacji == 'sigmoid':
            return 1 / (1 + np.exp(-np.clip(x, -250, 250)))  # Clip dla stabilności numerycznej
        elif self.funkcja_aktywacji == 'relu':
            return np.maximum(0, x)
        elif self.funkcja_aktywacji == 'tanh':
            return np.tanh(x)
        else:
            return x  # Linear
    
    def pochodna_aktywacji(self, x):
        if self.funkcja_aktywacji == 'sigmoid':
            s = self.aktywacja(x)
            return s * (1 - s)
        elif self.funkcja_aktywacji == 'relu':
            return np.where(x > 0, 1, 0)
        elif self.funkcja_aktywacji == 'tanh':
            return 1 - np.tanh(x)**2
        else:
            return np.ones_like(x)  # Linear
    
    def forward(self, X):
        # Dodajemy bias
        if X.ndim == 1:
            X = X.reshape(1, -1)
        X_z_biasem = np.hstack([np.ones((X.shape[0], 1)), X])
        
        self.wejscie = X_z_biasem
        self.net = X_z_biasem @ self.wagi.T
        self.wyjscie = self.aktywacja(self.net)
        
        return self.wyjscie
    
    def backward(self, delta_nastepna, stala_uczenia):
        # delta_nastepna: błąd z następnej warstwy
        delta = delta_nastepna * self.pochodna_aktywacji(self.net)
        
        # Oblicz gradienty wag
        dW = delta.T @ self.wejscie / self.wejscie.shape[0]
        
        # Aktualizuj wagi
        self.wagi -= stala_uczenia * dW
        
        # Oblicz błąd do przekazania do poprzedniej warstwy (bez biasu)
        delta_prev = delta @ self.wagi[:, 1:]
        
        return delta_prev

class SiecNeuronowa:
    def __init__(self, architektura, funkcje_aktywacji=None):
        self.warstwy = []
        
        if funkcje_aktywacji is None:
            funkcje_aktywacji = ['sigmoid'] * (len(architektura) - 1)
        
        for i in range(len(architektura) - 1):
            warstwa = Warstwa(architektura[i], architektura[i+1], funkcje_aktywacji[i])
            self.warstwy.append(warstwa)
    
    def forward(self, X):
        wyjscie = X
        for warstwa in self.warstwy:
            wyjscie = warstwa.forward(wyjscie)
        return wyjscie
    
    def backward(self, y, stala_uczenia):
        # Oblicz błąd na wyjściu
        wyjscie_ost = self.warstwy[-1].wyjscie
        delta = wyjscie_ost - y
        
        # Propagacja wsteczna przez wszystkie warstwy
        for i in range(len(self.warstwy) - 1, -1, -1):
            delta = self.warstwy[i].backward(delta, stala_uczenia)
    
    def fit(self, X, y, epoki=100, stala_uczenia=0.1, rozmiar_batcha=32, verbose=True):
        n_przykladow = X.shape[0]
        historia_straty = []
        
        for epoka in range(epoki):
            # Tasowanie danych
            indeksy = np.random.permutation(n_przykladow)
            X_tasowane = X[indeksy]
            y_tasowane = y[indeksy]
            
            strata_epoki = 0
            
            for i in range(0, n_przykladow, rozmiar_batcha):
                koniec = min(i + rozmiar_batcha, n_przykladow)
                X_batch = X_tasowane[i:koniec]
                y_batch = y_tasowane[i:koniec]
                
                # Forward propagation
                wyjscie = self.forward(X_batch)
                
                # Oblicz stratę (mean squared error)
                strata = np.mean((wyjscie - y_batch) ** 2)
                strata_epoki += strata * (koniec - i)
                
                # Backward propagation
                self.backward(y_batch, stala_uczenia)
            
            strata_epoki /= n_przykladow
            historia_straty.append(strata_epoki)
            
            if verbose and epoka % 10 == 0:
                print(f"Epoka {epoka}, Strata: {strata_epoki:.4f}")
        
        return historia_straty
    
    def predict(self, X):
        wyjscie = self.forward(X)
        return np.argmax(wyjscie, axis=1)
    
    def accuracy(self, X, y):
        przewidywania = self.predict(X)
        prawdziwe_etykiety = np.argmax(y, axis=1)
        return np.mean(przewidywania == prawdziwe_etykiety)

# Funkcje do ładowania danych MNIST
def load_mnist():
    """Ładuje dane MNIST"""
    try:
        with gzip.open('mnist.pkl.gz', 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik mnist.pkl.gz nie został znaleziony.")
        print("Proszę pobrać dane MNIST i umieścić je w tym samym katalogu.")
        return None, None, None

def przygotuj_dane(train_set, test_set):
    """Przygotowuje dane do treningu"""
    X_train, y_train = train_set
    X_test, y_test = test_set
    
    # Normalizacja pikseli do zakresu [0, 1]
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    
    # Konwersja etykiet na one-hot encoding
    y_train_onehot = np.eye(10)[y_train]
    y_test_onehot = np.eye(10)[y_test]
    
    return X_train, X_test, y_train_onehot, y_test_onehot

# Eksperymenty z różnymi architekturami
def eksperymentuj():
    train_set, valid_set, test_set = load_mnist()
    if train_set is None:
        return
    
    X_train, X_test, y_train, y_test = przygotuj_dane(train_set, test_set)
    
    # Różne architektury do przetestowania
    architektury = [
        [784, 128, 10],           # 1 warstwa ukryta, 128 neuronów
        [784, 256, 10],           # 1 warstwa ukryta, 256 neuronów  
        [784, 64, 64, 10],        # 2 warstwy ukryte, 64 neurony każda
        [784, 128, 64, 10],       # 2 warstwy ukryte, różne rozmiary
        [784, 512, 256, 10],      # 2 warstwy ukryte, większe
    ]
    
    for i, architektura in enumerate(architektury):
        print(f"\n--- Eksperyment {i+1}: {architektura} ---")
        
        # Użyj ReLU dla warstw ukrytych, softmax nie jest potrzebny z MSE
        funkcje_aktywacji = ['relu'] * (len(architektura) - 2) + ['sigmoid']
        
        siec = SiecNeuronowa(architektura, funkcje_aktywacji)
        
        print("Trenowanie...")
        historia = siec.fit(X_train, y_train, epoki=50, stala_uczenia=0.1, rozmiar_batcha=64)
        
        dokladnosc_train = siec.accuracy(X_train, y_train)
        dokladnosc_test = siec.accuracy(X_test, y_test)
        
        print(f"Dokładność na zbiorze treningowym: {dokladnosc_train:.4f}")
        print(f"Dokładność na zbiorze testowym: {dokladnosc_test:.4f}")

if __name__ == "__main__":
    eksperymentuj()